# Install dependencies

Download and Set Up Ollama (downloads the Ollama installation script, executes it and starts the Ollama server as a background process.)

In [13]:
import os
import asyncio

async def run_command_async(cmd):
  """Runs a shell command asynchronously."""
  process = await asyncio.create_subprocess_shell(
      cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE
  )
  stdout, stderr = await process.communicate()
  if process.returncode != 0:
    print(f"Error running command: {cmd}")
    print(f"Stderr: {stderr.decode()}")
  else:
    print(f"Successfully ran: {cmd}")

async def setup_ollama():
  """Downloads, installs, and starts the Ollama server."""
  await run_command_async("curl -fsSL https://ollama.com/install.sh | sh")
  # Start Ollama in the background using nohup to keep it running
  os.system('nohup ollama serve > ollama.log 2>&1 &')
  # Give the server a moment to initialize
  await asyncio.sleep(5)
  print("✅ Ollama server started in the background.")

# Run the asynchronous setup
await setup_ollama()

Successfully ran: curl -fsSL https://ollama.com/install.sh | sh
✅ Ollama server started in the background.


Install all the necessary Python libraries for the project.

In [2]:
!pip install gradio requests ollama -q

# Create Project files



* **config.py**: A list of models (to be used for the dropdown)
* **ollama_utils.py**: This module handles all direct interactions with the Ollama service.
* **github_utils.py**: This module is responsible for fetching content from GitHub.
* **llm_analyzer.py**: This module contains the core logic for interacting with the LLM to perform the analysis.
* **app_core.py**: the main entry point of the application. Imports other modules, defines the UI, and orchestrate the logic.

`config.py: A list of models (to be used for the dropdown)`

In [3]:
# --- Create config.py ---
%%writefile config.py

# A curated list of popular and effective models for the dropdown
CURATED_MODEL_LIST = [
    "llama3:8b",          # Meta's latest 8B model (Default)
    "gemma3:270M",         # google Gemma 270M
    "gpt-oss:20b",        # OpenAI’s open-weight models designed for powerful reasoning, agentic tasks, and versatile developer use cases.
    "deepseek-coder:6.7b",# A top-tier coding model.
    "mistral:7b",         # Mistral's popular 7B model
    "gemma2:9b",          # Google's latest 9B model
    "phi3:3.8b",          # Microsoft's small, powerful model
    "qwen2:7b",           # Alibaba's latest model
    "codellama:7b",       # Code-specialized model
    "starcoder2:3b",      # Another code model from HuggingFace/ServiceNow
    "sqlcoder:7b",        # Specialized for SQL generation
    "tinydolphin:1.1b"    # A very small, fast model for quick tests
]


Writing config.py


`ollama_utils.py: This module handles all direct interactions with the Ollama service.`


In [4]:
# --- Create ollama_utils.py ---
%%writefile ollama_utils.py

import ollama
from typing import List

# This module handles all direct interactions with the Ollama service.

def get_local_models() -> List[str]:
    """Gets the list of models already pulled locally."""
    try:
        models_info = ollama.list()
        return [model['model'] for model in models_info['models']]
    except Exception:
        return []

def ensure_model_is_pulled(model_name: str):
    """Checks if a model is available locally, and pulls it if not."""
    local_models = get_local_models()
    if model_name not in local_models:
        try:
            print(f"Model '{model_name}' not found locally. Pulling from Ollama...")
            ollama.pull(model_name)
            print("Model pulled successfully.")
        except Exception as e:
            # Raise a standard error that the main app function will catch and display.
            raise RuntimeError(f"Failed to pull model '{model_name}'. Please check the model name and your connection. Details: {e}")

Writing ollama_utils.py


`github_utils.py: This module is responsible for fetching content from GitHub.`

In [5]:
# --- Create github_utils.py ---
%%writefile github_utils.py

# This module is responsible for fetching content from GitHub.
import requests

def get_readme_content(github_url: str) -> str:
    """Fetches and cleans the README content from a GitHub repo URL."""
    try:
        # To get the raw README, we can adjust the URL
        # e.g., https://github.com/user/repo -> https://raw.githubusercontent.com/user/repo/main/README.md
        if "github.com" not in github_url:
            return "Error<repoAnalyzerAgent>: Please provide a valid GitHub URL."

        parts = github_url.strip("/").split("/")
        user = parts[-2]
        repo = parts[-1]

        # Try common main branch names
        for branch in ["main", "master","develop", "dev"]:
            raw_url = f"https://raw.githubusercontent.com/{user}/{repo}/{branch}/README.md"
            response = requests.get(raw_url)
            if response.status_code == 200:
                return response.text

        return "Error<repoAnalyzerAgent>: Could not find README.md in 'main' or 'master' branch."

    except Exception as e:
        return f"Error<repoAnalyzerAgent> fetching content: {e}"




Writing github_utils.py


`llm_analyzer.py: This module contains the core logic for interacting with the LLM to perform the analysis.`

In [6]:
# --- Create llm_analyzer.py ---
%%writefile llm_analyzer.py

#This module contains the core logic for interacting with the LLM to perform the analysis.
import ollama
import time
from typing import Tuple

def analyze_repo_with_llm(context: str, model_name: str) -> Tuple[str, str]:
    """Uses a local LLM via Ollama to analyze the repo content."""

    prompt = f"""
    You are an expert senior AI researcher with 20 years of experience.
    Your task is to analyze the following GitHub repository's README file and provide a structured, concise, and insightful summary.

    **README Content:**
    ---
    {context}
    ---

    **Your Analysis:**
    Provide the output in the following markdown format:

    ### 🚀 Project Summary
    (A brief, one-paragraph summary of the project's main goal and functionality.)

    ### 🛠️ Key Technologies & Libraries
    (A bulleted list of the primary technologies, languages, and libraries mentioned or implied.)

    ### 💡 Potential Use Cases
    (A bulleted list of 2-3 potential real-world applications for this project.)

    ### 📈 Complexity
    (Your expert opinion on the project's complexity: Beginner, Intermediate, or Advanced.)
    """

    try:
        print("-> Sending request to LLM...")
        start_time = time.time() # Start the timer
        response = ollama.chat(
            model= model_name,  # Or 'gemma2', 'llamma3', 'gemma3:1B', 'gemma3:270M'
            messages=[{'role': 'user', 'content': prompt}]
        )
        end_time = time.time() # End the timer
        print("-> Received response from LLM.")
        duration = end_time - start_time
        llm_response = response['message']['content']
        # Append the timing information to the response for display
        timing_info = f"*LLM processing time: {duration:.2f} seconds.*"

        print("-> [*] "+timing_info[1:-1])
        return llm_response, timing_info

    except Exception as e:
        print(f"Error communicating with LLM: {e})")
        return f"Error <repoAnalyzerAgent> communicating with LLM {model_name}: {e}", ""

Writing llm_analyzer.py


`app_core.py: the main entry point of the application. Imports other modules, defines the UI, and orchestrate the logic.`

In [14]:
# --- Create app_core.py ---
%%writefile app_core.py

# This is the main entry point of the application. Imports other modules, defines the UI, and orchestrate the logic.
import gradio as gr
from typing import Tuple
import argparse

# Import functions from our other modules
from config import CURATED_MODEL_LIST
from ollama_utils import ensure_model_is_pulled
from github_utils import get_readme_content
from llm_analyzer import analyze_repo_with_llm



def analyze_github_repo(url: str, model_name: str, progress = gr.Progress(track_tqdm=True)) -> Tuple[str, str]:
    """The main function that ties everything together for the UI."""
    if not url or not model_name:
        return "Please enter a GitHub repository URL to begin and select a model.", "Please enter a GitHub repository URL to begin and select a model."

    # step 0: start
    msg_0 = "Initiating github repo anlysis..."
    if isinstance(progress, gr.Progress):
        progress(0, desc=msg_0)
    else:
        print(msg_0)


    # Step 1: Ensure the model is available
    msg_1 = f"Step (1/4) Checking model: {model_name}..."
    try:
        if isinstance(progress, gr.Progress):
            progress(0.25, desc=msg_1)
        else:
            print(msg_1)
        ensure_model_is_pulled(model_name)
    except (gr.Error, RuntimeError) as e:
        return str(e), str(e)

    # Step 2: Fetch README content
    msg_2 = f"Step (2/4) Fetching content for: {url}..."
    if isinstance(progress, gr.Progress):
        progress(0.5, desc=msg_2)
    else:
        print(msg_2)
    readme_content = get_readme_content(url)
    if "Error<repoAnalyzerAgent>" in readme_content:
        return readme_content, readme_content

    # Step 3: Analyze with the LLM
    msg_3 = f"Step (3/4) Analyzing content with LLM -> {model_name}..."
    if isinstance(progress, gr.Progress):
        progress(0.75, desc=msg_3)
    else:
        print(msg_3)
    analysis, timing_info = analyze_repo_with_llm(readme_content, model_name)
    #print(timing_info)

    # Step 4: Done
    msg_4 = "Step (4/4) Done!"
    if isinstance(progress, gr.Progress):
        progress(1, desc= msg_4)
    else:
        print(msg_4)
    return analysis, timing_info


def clear_fields():
    """Returns empty values to clear all input and output fields."""
    # Clears: url_input, analysis_output, time_output, model_dropdown
    return "", "", "", CURATED_MODEL_LIST[0]

def create_ui():
    """Creates the layout and returns the Gradio UI Blocks"""
    with gr.Blocks() as iface:
        gr.Markdown("# 🤖 GitHub Repo Analyzer Agent")
        gr.Markdown("Enter a GitHub repo URL to get an expert analysis from a local LLM. The first analysis with a model may take some time as the model loads into memory.")

        with gr.Row():
            with gr.Column(scale=3):
                url_input = gr.Textbox(
                    lines=15,
                    placeholder="e.g., https://github.com/ollama/ollama",
                    label="GitHub Repository URL"
                )
            with gr.Column(scale=1):
                model_dropdown = gr.Dropdown(
                    choices=CURATED_MODEL_LIST,
                    value=CURATED_MODEL_LIST[0],
                    label="Select or Enter a LLM Model Name",
                    allow_custom_value=True
                )
                submit_btn = gr.Button("Submit", variant="primary")
                stop_btn = gr.Button("Stop", variant="stop")
                clear_btn = gr.Button("Clear")
                time_output = gr.Textbox(label="Processing information", elem_id="time-output")

        with gr.Row():
            analysis_output = gr.Markdown(label="Expert Analysis", elem_id="analysis-output")

        gr.Examples(
            examples=[
                ["https://github.com/openai/gpt-oss"],
                ["https://github.com/facebookresearch/llama"],
                ["https://github.com/huggingface/transformers"],
                ["https://github.com/matlab-deep-learning/llms-with-matlab"],
            ],
            inputs=url_input
        )

        # Event handling
        analysis_event = submit_btn.click(
            fn=analyze_github_repo,
            inputs=[url_input, model_dropdown],
            outputs=[analysis_output, time_output]
        )
        stop_btn.click(fn=None, inputs=None, outputs=None, cancels=[analysis_event])
        clear_btn.click(fn=clear_fields, inputs=None, outputs=[url_input, analysis_output, time_output, model_dropdown])

    return iface


def main_cli():
    """Handles the command-line interface logic."""
    parser = argparse.ArgumentParser(description="GitHub Repo Analyzer CLI")
    parser.add_argument("--url", required=True, type=str, help="URL of the GitHub repository to analyze.")
    parser.add_argument("--model", type=str, default=CURATED_MODEL_LIST[0], help=f"Name of the Ollama model to use (default: {CURATED_MODEL_LIST[0]}).")
    args = parser.parse_args()

    print("--- GitHub Repo Analyzer CLI ---")
    analysis, timing_info = analyze_github_repo(args.url, args.model, progress=None)
    print("\n--- Analysis Result ---")
    print(analysis)
    print("-----------------------")
    print(f"Processing Time: {timing_info}")
    print("-----------------------")



if __name__ == "__main__":
    import sys
    # Check if any command-line arguments for the CLI were provided
    if len(sys.argv) > 1 and any(arg.startswith('--') for arg in sys.argv):
        main_cli()
    else:
        # If no CLI arguments, launch the Gradio UI
        app_ui = create_ui()
        print("Launching Gradio Interface...")
        app_ui.launch(debug=True, share=False)


Overwriting app_core.py


# Prep-pull a default Language Model

Pull a default model (e.g llama3:8b) to ensure the app is ready to use immediately after setup. This may take a few minutes.

In [8]:
import ollama

try:
    print("⏳ Pulling the default model (llama3:8b)...")
    ollama.pull('llama3:8b')
    print("✅ Default model pulled successfully!")
except Exception as e:
    print(f"❗️ An error occurred while pulling the model: {e}")
    print("📄 Check the ollama.log for more details by running !cat ollama.log")

⏳ Pulling the default model (llama3:8b)...
✅ Default model pulled successfully!


# Run the repo analyzer application

Execute the main Python script to launch the Gradio app.

In [ ]:
!python app_core.py

Launching Gradio Interface...
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
